In [ ]:
import rmgpy
from rmgpy.rmg.model import CoreEdgeReactionModel
from rmgpy.data.rmg import RMGDatabase
from rmgpy.rmg.react import *
from rmgpy.reaction import Reaction
from rmgpy.molecule.molecule import Molecule
from rmgpy.molecule.resonance import *
from rmgpy.species import Species
from rmgpy.thermo.thermoengine import submit
from rmgpy.kinetics.kineticsdata import KineticsData
from rmgpy.data.kinetics.family import TemplateReaction
from rmgpy.data.kinetics.depository import DepositoryReaction

from IPython.display import display

In [ ]:
family = 'R_Addition_MultipleBond'

In [ ]:
databasePath = rmgpy.settings['database.directory']

database = RMGDatabase()
database.load(
    path = databasePath,
    thermoLibraries = ['primaryThermoLibrary'],
    reactionLibraries = [],
    seedMechanisms = [],
    kineticsFamilies = [family],
)

for family in database.kinetics.families.itervalues():
    family.addKineticsRulesFromTrainingSet(thermoDatabase=database.thermo)
    family.fillKineticsRulesByAveragingUp(verbose=True)

In [ ]:
reactants = (
    Species().fromSMILES("c1ccccc1"),
    Species().fromSMILES("[c]1ccccc1"),
)
for r in reactants:
    submit(r)

In [ ]:
for r in reactants:
    for m in r.molecule:
        display(m)

In [ ]:
result = reactSpecies(reactants)

In [ ]:
cerm = CoreEdgeReactionModel()
for rxn0 in result:
    rxn1 = cerm.makeNewReaction(rxn0)
for rxn0 in cerm.newReactionList:
    cerm.applyKineticsToReaction(rxn0)
    if isinstance(rxn0.kinetics, KineticsData):
        rxn0.kinetics = reaction.kinetics.toArrhenius()
    if isinstance(rxn0,TemplateReaction) or isinstance(rxn0,DepositoryReaction):
        rxn0.fixBarrierHeight()

In [ ]:
for rxn0 in cerm.newReactionList:
    display(rxn0)
    print rxn0.template
    print rxn0.kinetics